In [2]:
import ipywidgets as widgets
from IPython.display import display, HTML
from tvbwidgets.ui.connectivity_matrix_editor_widget import ConnectivityMatrixEditor
from tvbwidgets.ui.spacetime_widget import SpaceTimeVisualizerWidget
from tvb.datatypes.connectivity import Connectivity
from ipywidgets import HTML
import numpy as np


# Load connectivity data
conn = Connectivity.from_file()
conn.configure()


# ---- Custom SpaceTime Visualizer ----
class CustomSpaceTimeEditor(SpaceTimeVisualizerWidget, widgets.VBox):
    def __init__(self, connectivity, width=600, height=600):
        SpaceTimeVisualizerWidget.__init__(self, connectivity, width=width, height=height)
        widgets.VBox.__init__(self)

        self.conduction_speed = self.options.children[0].value
        self.update_plot_details = HTML(
            value=self.update_generate_details(connectivity.tract_lengths.max()),
            layout=self.DEFAULT_BORDER
        )

        self.children = [widgets.Output(), self.update_plot_details]

        with self.children[0]:
            self.display()

    def on_update_tract_length(self, new_max_tract_length,new_min_tract_length):
        self.graphs_matplotlib.clear_output(wait=True)
        self.conduction_speed = self.options.children[0].value

        # Update slider limits
        self.options.children[1].max = new_max_tract_length / self.conduction_speed
        self.options.children[2].max = new_max_tract_length / self.conduction_speed
        self.options.children[1].min = new_min_tract_length / self.conduction_speed
        self.options.children[2].min = new_min_tract_length / self.conduction_speed
        self.options.children[1].value = self.options.children[1].min
        self.options.children[2].value = self.options.children[2].max
        self.from_time = self.options.children[1].value
        self.to_time = self.options.children[2].value

        # Update plot detail text
        self.update_plot_details.value = self.update_generate_details(new_max_tract_length)

        # Redraw all slices
        for i in range(len(self.graph_slices)):
            connectivity = self._prepare_connectivity(i)
            data = self._generate_colors(connectivity)
            self.graph_slices[i].material.map.data = np.fliplr(data)
            self.ims[i].imshow(data)

        with self.graphs_matplotlib:
            display(self.fig)

    def update_generate_details(self, new_max_tract_length):
        return f"""<br>
            <div style="line-height:1px;">
            <br><h3>PLOT DETAILS</h3> <br><br>
            <h4>Conduction Speed:</h4><br> 
            {self.conduction_speed} mm/ms<br>
            <h4>Min (non-zero) Delay:</h4><br>
            {np.min(self.connectivity.tract_lengths[np.nonzero(self.connectivity.tract_lengths)]) / self.conduction_speed:.2f} ms<br>
            <h4>Max Delay:</h4><br>
            {new_max_tract_length / self.conduction_speed:.2f} ms<br>
            <h4>Min (non-zero) Tract Length:</h4><br>
            {np.min(self.connectivity.tract_lengths[np.nonzero(self.connectivity.tract_lengths)])} mm<br>
            <h4>Max Tract Length:</h4><br>
            {new_max_tract_length} mm<br>
            <h4>Min (non-zero) Weight:</h4><br>
            {np.min(self.connectivity.weights[np.nonzero(self.connectivity.weights)])}<br>
            <h4>Max Weight:</h4><br>
            {np.max(self.connectivity.weights)}<br><br></div>"""


# ---- Custom Connectivity Editor ----
class CustomMatrixEditor(widgets.VBox, ConnectivityMatrixEditor):
    def __init__(self, connectivity, viewer, **kwargs):
        ConnectivityMatrixEditor.__init__(self, connectivity, **kwargs)
        widgets.VBox.__init__(self)
        self.viewer = viewer
        self.children = [widgets.Output()]
        with self.children[0]:
            self.display()

        # Optional: Add reset button (uncomment below to activate)
        # self.reset_button = widgets.Button(description="Reset")
        # self.reset_button.on_click(self.reset_matrix)
        # self.children += [self.reset_button]

    def on_apply_change(self, change):
        super().on_apply_change(change)
        if hasattr(self, "new_max_tract_length") and self.new_max_tract_length is not None:
            self.viewer.on_update_tract_length(self.new_max_tract_length,self.new_min_tract_length)
        if hasattr(self,"new_min_tract_length") and self.new_min_tract_length is not None:
            self.viewer.on_update_tract_length(self.new_max_tract_length,self.new_min_tract_length)

    # Optional: Reset method if needed
    # def reset_matrix(self, _):
    #     self.connectivity = Connectivity.from_file()
    #     self.connectivity.configure()
    #     self.viewer.connectivity = self.connectivity
    #     self.new_max_tract_length = self.connectivity.tract_lengths.max()
    #     self.viewer.on_update_max_tract_length(self.new_max_tract_length)
    #     self.display()  # redisplay the matrix


# ---- Instantiate and Display ----
viewer = CustomSpaceTimeEditor(connectivity=conn)
editor = CustomMatrixEditor(connectivity=conn, viewer=viewer)

display(viewer)
display(editor)


2025-04-05 06:35:43,891 - WARNING - tvb.basic.readers - File 'hemispheres' not found in ZIP.


CustomSpaceTimeEditor(children=(Output(), HTML(value='<br>\n            <div style="line-height:1px;">\n      …

CustomMatrixEditor(children=(Output(),))